In [1]:
with open('input') as f:
    data = f.read().strip()

In [2]:
from string import ascii_lowercase

parsed = [list(line) for line in data.split('\n')]

# get x and y coordinates for start and goal
from itertools import chain
flatten = list(chain(*parsed))

x_start = flatten.index('S') % len(parsed[0])
y_start = flatten.index('S') // len(parsed[0])
start = (x_start, y_start)

x_end = flatten.index('E') % len(parsed[0])
y_end = flatten.index('E') // len(parsed[0])
end = (x_end, y_end)

# validate
assert parsed[y_start][x_start] == 'S'
assert parsed[y_end][x_end] == 'E'

# create height matrix

parsed[y_start][x_start] = 'a'
parsed[y_end][x_end] = 'z'

letter_vals = {y:x for x,y in enumerate(ascii_lowercase)}
heights = [[letter_vals[entry] for entry in line] for line in parsed]


In [3]:
from queue import PriorityQueue

class Graph():

    def __init__(self, height_matrix):
        self.nodes = []
        self.edges = []

        for y1, line in enumerate(height_matrix):
            for x1, entry in enumerate(line):
                
                self.nodes.append((x1,y1))
                h_xy1 = heights[y1][x1]

                if x1 > 0:
                    x2, y2 = x1-1, y1
                    h_xy2 = heights[y2][x2]
                    diff = h_xy2 - h_xy1
                    self.edges.append(
                        ((x1,y1), (x2,y2), diff)
                    )
                if x1 < len(line)-1:
                    x2, y2 = x1+1, y1
                    h_xy2 = heights[y2][x2]
                    diff = h_xy2 - h_xy1
                    self.edges.append(
                        ((x1,y1), (x2,y2), diff)
                    )
                if y1 > 0:
                    x2, y2 = x1, y1-1
                    h_xy2 = heights[y2][x2]
                    diff = h_xy2 - h_xy1
                    self.edges.append(
                        ((x1,y1), (x2,y2), diff)
                    )
                if y1 < len(height_matrix)-1:
                    x2, y2 = x1, y1+1
                    h_xy2 = heights[y2][x2]
                    diff = h_xy2 - h_xy1
                    self.edges.append(
                        ((x1,y1), (x2,y2), diff)
                    )
        
        self.traversable = [e for e in self.edges if e[2] <= 1]
        self.r_traversable = [e for e in self.edges if (e[2]*-1) <= 1]

    def dijkstra(self, start, reverse_direction=False):
        D = {n:float('inf') for n in self.nodes}
        D[start] = 0
        visited = []

        edges = self.r_traversable if reverse_direction else self.traversable

        pq = PriorityQueue()
        pq.put((0, start))

        while not pq.empty():
            (dist, current_node) = pq.get()
            visited.append(current_node)

            for xy1, xy2, _ in edges:
                if xy1 == current_node:
                    if xy2 not in visited:
                        old_cost = D[xy2]
                        new_cost = D[current_node] + 1
                        if new_cost < old_cost:
                            pq.put((new_cost, xy2))
                            D[xy2] = new_cost
        
        self.D = D

# Part 1

In [4]:
g = Graph(parsed)
g.dijkstra(start)
distance = g.D[end]
print(f'Distance from start to end is {distance}.')

Distance from start to end is 447.


# Part 2

In [5]:
flatten = list(chain(*parsed))

starting_points = []

for i, e in enumerate(flatten):
    if e == 'a':
        x_start = i % len(parsed[0])
        y_start = i // len(parsed[0])
        starting_points.append((x_start, y_start))

print(f'{len(starting_points)} starting points')

2092 starting points


In [6]:
g.dijkstra(end, reverse_direction=True)

dist_to_start = [(start, g.D[start]) for start in starting_points]
closest_point, distance = sorted(dist_to_start, key=lambda x: x[1])[0]
print(f'closest distance is {distance} for starting point {closest_point}.')

closest distance is 446 for starting point (0, 19).
